# Secure Multi-Party Voting

In this notebook,
we explore a voting protocol which uses secure multi-party computation (SMPC)
to aggregate votes,
without individual votes being discoverable.
This protocol is best suited to a voting session with well-defined,
non-cooperating parties,
such as political parties in a general election.
Each party is given a share of a vote,
and computes a sum over their shares.
Finally,
parties combine their encrypted shares and decrypt it
to reveal the final vote.
Please read [this PySyft tutorial](https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%2009%20-%20Intro%20to%20Encrypted%20Programs.ipynb)
for more information on SMPC.

*THIS IS A POC. SSI/SECURE COMMUNICATION IS NOT IN PLACE.*

For this POC,
we shall perform a categorical vote on "Best PriCon workshop".

### Assumptions of this protocol:
- Limited, well-defined vote options
    - Vote values (e.g. "PryVote") must be converted into an integer to be shared between parties. To assign each vote value to an integer, we need to know how many possible vote values there are ahead of time
- Binary voting
    - Only two options supported at this time
- Majority vote scheme
    - Quadratic voting not supported
    - "multi-vote" schemes, such as STV, not supported

In [34]:
import random
import uuid

In [50]:
vote_values = {
    "PryVote": 0,
    "PyDP": 1,
}

vote_classes = {v: k for k, v in vote_values.items()}

## Define roles

We create simple _Voter_ and _Party_ classes.
Voter:
- Given a unique ID
- Can store a personal vote value
- Can send vote and ID to parties
Party:
- Can aggregate votes
- Can share vote aggregates amongst other parties

In [42]:
class Voter:
    def __init__(self, Q) -> None:
        self._id = uuid.uuid4()
        self._Q = Q
        self._vote_shares = None

    def update_vote(self, vote_value: str) -> None:
        try:
            vote_class = vote_values[vote_value] 
            self._vote_shares = self._encrypt_vote(vote_class)
        except KeyError:
            self._vote_shares = None

    def _encrypt_vote(self, vote: int):
        share_a = random.randint(-Q,Q)
        share_b = random.randint(-Q,Q)
        share_c = (vote - share_a - share_b) % Q
        return (share_a, share_b,  share_c)


    def send_vote(self, parties) -> None:
        if self._vote_shares is None:
            print(f"{self._id} has not set a vote")
            return

        assert len(self._vote_shares) == len(parties)

        for vote_share, party in zip(self._vote_shares, parties):
            party.receive_vote(self._id, vote_share)

In [36]:
class Party:
    def __init__(self, name: str, Q: int) -> None:
        self._name = name
        self._Q = Q
        self._ids = []
        self._vote_sum = 0

    def receive_vote(self, voter_id: uuid.UUID, vote: int) -> None:
        if voter_id not in self._ids:
            print(f"{self._name}: Adding vote for {voter_id}")
            self._ids.append(voter_id)
            self._vote_sum += vote
        else:
            print(f"{self._name}: {voter_id} has already voted")

## Define Q

TODO what Q is

In [37]:
Q = 1234567891011

## Create Voters and Vote counters

5 voters, 3 vote counters (red, blue, yellow)

In [43]:
red = Party("red", Q)
blue = Party("blue", Q)
yellow = Party("yellow", Q)

In [44]:
alice = Voter(Q)
bob = Voter(Q)
charlie = Voter(Q)
dan = Voter(Q)
eve = Voter(Q)

## Vote

In [45]:
alice.update_vote("PryVote")
bob.update_vote("PryVote")
charlie.update_vote("PyDP")
dan.update_vote("PryVote")
eve.update_vote("PyDP")

In [46]:
for voter in [alice, bob, charlie, dan, eve]:
    voter.send_vote([red, blue, yellow])

red: Adding vote for b9cb3d45-cf17-46ca-8407-d9ad2ded1c67
blue: Adding vote for b9cb3d45-cf17-46ca-8407-d9ad2ded1c67
yellow: Adding vote for b9cb3d45-cf17-46ca-8407-d9ad2ded1c67
red: Adding vote for a1e67a55-10e5-4edd-be4f-0905608bc895
blue: Adding vote for a1e67a55-10e5-4edd-be4f-0905608bc895
yellow: Adding vote for a1e67a55-10e5-4edd-be4f-0905608bc895
red: Adding vote for b0b27cfb-022a-4491-ac9a-9407f03fc2d2
blue: Adding vote for b0b27cfb-022a-4491-ac9a-9407f03fc2d2
yellow: Adding vote for b0b27cfb-022a-4491-ac9a-9407f03fc2d2
red: Adding vote for 39acd924-c832-443b-8109-22b9a155aeb4
blue: Adding vote for 39acd924-c832-443b-8109-22b9a155aeb4
yellow: Adding vote for 39acd924-c832-443b-8109-22b9a155aeb4
red: Adding vote for fc6e2053-f9af-453b-a461-359afd937c81
blue: Adding vote for fc6e2053-f9af-453b-a461-359afd937c81
yellow: Adding vote for fc6e2053-f9af-453b-a461-359afd937c81


## Decrypt the Vote

In [53]:
total_sum = (red._vote_sum + blue._vote_sum + yellow._vote_sum) % Q
average_vote = total_sum / len(red._ids)

print(f"Average vote is {average_vote}; Therefore {vote_classes[round(average_vote)]} is the winner!")

Average vote is 0.4; Therefore PryVote is the winner!


---

## Evaluation of the protocol

#### The Voter

If a sensible final vote was produced,
each voter knows that their vote was counted correctly by all parties,
_or_ that all parties did not count their vote.
Collusion between all parties is unlikely in adversarial contexts,
such as elections.
In less combative vote sessions (including this pretend setting!),
where there are no clearly opposing parties,
collusion is more likely and the voters might place less trust in this protocol. 

- Vote parties do not know what each person voted

#### Vote parties
- Vote parties each have a stake in the vote; do not have to trust a singular authority

However,
a vote party who is fairly sure they are going to lose
could invalidate the vote
by falsifying vote shares.
Under this protocol we **cannot identify which party made the "mistake"**.
This could be solved by policy,
such as frequent, independent auditing of a running vote aggregation during the lifetime of a vote session.